# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [21]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
import re
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/mohammad/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/mohammad/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mohammad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('cleaned_table', engine)
                      
X = df['message']
y = df.iloc[:,4:]
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [3]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

wnl = WordNetLemmatizer()

def tokenize(text):
    cleaned_text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    cleaned_text = word_tokenize(cleaned_text)
    cleaned_text = [w for w in cleaned_text if w not in stopwords.words("english")]
    return [wnl.lemmatize(x) for x in cleaned_text]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import AdaBoostClassifier

class MyTfidfVectorizer(TfidfVectorizer):
    def fit_transform(self, X, y):
        result = super(MyTfidfVectorizer, self).fit_transform(X, y)
        result.sort_indices()
        return result

pipeline = Pipeline([
    ('tfidf_vectorizer', MyTfidfVectorizer(tokenizer=tokenize)),
    ('classifier', MultiOutputClassifier(AdaBoostClassifier(), n_jobs=-1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X,y)

model = pipeline.fit(X_train, y_train)

KeyboardInterrupt: 

In [40]:
y_preds = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [44]:
from sklearn.metrics import accuracy_score, classification_report, precision_score

y_test_numpy = y_test.to_numpy()
y_preds = y_preds.astype(int)

for i, column_name in zip(range(36), y_test.columns):
    print(f'''Column_name: {column_name}
    {classification_report(y_test_numpy[:,i], y_preds[:,i])}
    --------------------------------------------------------------------------''')
print(f'''Aggregated report
    {classification_report(y_test_numpy.flatten(), y_preds.flatten())}''')

Column_name: related
                  precision    recall  f1-score   support

           0       0.72      0.16      0.26      1573
           1       0.79      0.98      0.87      4981

    accuracy                           0.78      6554
   macro avg       0.75      0.57      0.57      6554
weighted avg       0.77      0.78      0.73      6554

    --------------------------------------------------------------------------
Column_name: request
                  precision    recall  f1-score   support

           0       0.92      0.97      0.94      5502
           1       0.76      0.54      0.63      1052

    accuracy                           0.90      6554
   macro avg       0.84      0.75      0.79      6554
weighted avg       0.89      0.90      0.89      6554

    --------------------------------------------------------------------------
Column_name: offer
                  precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
      

Column_name: shops
                  precision    recall  f1-score   support

           0       1.00      1.00      1.00      6521
           1       0.62      0.15      0.24        33

    accuracy                           1.00      6554
   macro avg       0.81      0.58      0.62      6554
weighted avg       0.99      1.00      0.99      6554

    --------------------------------------------------------------------------
Column_name: aid_centers
                  precision    recall  f1-score   support

           0       0.99      1.00      0.99      6472
           1       0.48      0.16      0.24        82

    accuracy                           0.99      6554
   macro avg       0.74      0.58      0.62      6554
weighted avg       0.98      0.99      0.98      6554

    --------------------------------------------------------------------------
Column_name: other_infrastructure
                  precision    recall  f1-score   support

           0       0.96      0.99      0.97


### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
from sklearn.model_selection import GridSearchCV

parameters = {
    'classifier__estimator__n_estimators': [50, 100, 200],
    'classifier__estimator__learning_rate':[.01, .1, 1]
}

cv = GridSearchCV(pipeline, param_grid=parameters) 

model_cv = cv.fit(X_train, y_train)

/home/mohammad/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [28]:
cv.best_estimator_

Pipeline(memory=None,
         steps=[('tfidf_vectorizer',
                 MyTfidfVectorizer(analyzer='word', binary=False,
                                   decode_error='strict',
                                   dtype=<class 'numpy.float64'>,
                                   encoding='utf-8', input='content',
                                   lowercase=True, max_df=1.0,
                                   max_features=None, min_df=1,
                                   ngram_range=(1, 1), norm='l2',
                                   preprocessor=None, smooth_idf=True,
                                   stop_words=None, strip_accents=None,
                                   sublinear_tf=False,
                                   token_pattern='(?u)\\b\\w\\w+\\b',
                                   tokenizer=<function tokenize at 0x7f39756fab70>,
                                   use_idf=True, vocabulary=None)),
                ('classifier',
                 MultiOutputClassifie

In [45]:
y_preds_cv = model_cv.predict(X_test)

In [46]:
for i, column_name in zip(range(36), y_test.columns):
    print(f'''Column_name: {column_name}
    {classification_report(y_test_numpy[:,i], y_preds_cv[:,i])}
    --------------------------------------------------------------------------''')
    
print(f'''Aggregated report
    {classification_report(y_test_numpy.flatten(), y_preds_cv.flatten())}''')

Column_name: related
                  precision    recall  f1-score   support

           0       0.72      0.16      0.26      1573
           1       0.79      0.98      0.87      4981

    accuracy                           0.78      6554
   macro avg       0.75      0.57      0.57      6554
weighted avg       0.77      0.78      0.73      6554

    --------------------------------------------------------------------------
Column_name: request
                  precision    recall  f1-score   support

           0       0.92      0.97      0.94      5502
           1       0.76      0.54      0.63      1052

    accuracy                           0.90      6554
   macro avg       0.84      0.75      0.79      6554
weighted avg       0.89      0.90      0.89      6554

    --------------------------------------------------------------------------
Column_name: offer
                  precision    recall  f1-score   support

           0       1.00      1.00      1.00      6528
      

Column_name: storm
                  precision    recall  f1-score   support

           0       0.95      0.98      0.97      5913
           1       0.76      0.53      0.62       641

    accuracy                           0.94      6554
   macro avg       0.85      0.76      0.79      6554
weighted avg       0.93      0.94      0.93      6554

    --------------------------------------------------------------------------
Column_name: fire
                  precision    recall  f1-score   support

           0       0.99      1.00      0.99      6473
           1       0.69      0.30      0.41        81

    accuracy                           0.99      6554
   macro avg       0.84      0.65      0.70      6554
weighted avg       0.99      0.99      0.99      6554

    --------------------------------------------------------------------------
Column_name: earthquake
                  precision    recall  f1-score   support

           0       0.98      0.99      0.99      5987
      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [22]:
import pickle
pickle.dump(model_cv, open('classifier.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.